In [ ]:
# Required Libraries for EDA
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install seaborn
%pip install statistics
%pip install yfinance

In [11]:
import pandas as pd
import yfinance as yf
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:

#--------------------------------------
# importing libraries
#--------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xlsxwriter
from Demos.RegCreateKeyTransacted import del_trans
from IPython.core.pylabtools import figsize
from IPython.display import display
from matplotlib.pyplot import figure

#pip install xlsxwriter
#pip install IPython
import matplotlib
warnings.filterwarnings("ignore", category=matplotlib.MatplotlibDeprecationWarning)
import warnings
warnings.filterwarnings("ignore", category=MatplotlibDeprecationWarning)
warnings.filterwarnings("ignore", message="line style .* not understood")
warnings.filterwarnings("ignore", category=FutureWarning, message="No supported index is available.*")


# ==============================================================================
#  CONFIGURATION AND SETUP
# ==============================================================================
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
#-------------------------------------------------------------------------------

# ==============================================================================
# DATA IMPORT
# ==============================================================================
## 📁 importing Libyana LTE KPI Data
lte_rawdata = pd.read_excel('Datasets/Libyana Dataset JUN25/History Query-LTE KPI one year daily.xlsx',
                            sheet_name='Sheet0')
#Checking basic correlation
lte_rawdata.select_dtypes(include='number').corr().to_csv('exports/Libyana LTE KPIs/lte_rawdata_corr.csv')


## ==============================================================================
# ⚙️️ EDA summary function with Export feature
## ==============================================================================
def basic_EDA_summary(df, name="DataFrame"):
    print(f"\n ---- Head of {name}")
    display(df.head(10))
    print(f"\n ---- Shape of {name} dataframe")
    display(df.shape)
    print(f"\n ---- Data Types of {name}")
    display(df.dtypes)
    print(f"\n ---- Info of {name}")
    display(df.info())
    # Format export file path dynamically
    file_path = f"exports/Libyana LTE KPIs/{name}_eda_summary.xlsx"
    # Export sections to Excel sheets
    with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
        df.head(10).to_excel(writer, sheet_name="Head", index=False)
        pd.DataFrame({
            'Rows': [df.shape[0]],
            'Columns': [df.shape[1]]
        }).to_excel(writer, sheet_name="Shape", index=False)
        df.dtypes.reset_index().rename(columns={0: 'DataType', 'index': 'Column'}) \
            .to_excel(writer, sheet_name="DataTypes", index=False)
    print(f"\n✅ Exported EDA summary to {file_path}")

# ==============================================================================
# 🔍 Exploring lte raw data
## ==============================================================================
basic_EDA_summary(lte_rawdata, name="LTE Raw Data")

# 🔍 checking how many LTE enodeB
lte_rawdata['eNodeB Name'].unique()  # 16 eNodeBs
lte_rawdata['eNodeB Name'].nunique() # 16 eNodeBs
lte_rawdata['eNodeB Name'].value_counts() # Number of observations per eNodeB
# 🔍 checking how many LTE Cells
lte_rawdata['E-UTRAN FDD Cell Name'].unique()
lte_rawdata['E-UTRAN FDD Cell Name'].nunique()
lte_rawdata['E-UTRAN FDD Cell Name'].value_counts().to_csv('exports/noofcellsobsercations.csv')

# ==============================================================================
# 📊 Select KEY Radio Features required for forecasting PS traffic
# ==============================================================================
lte_ps_traffic = lte_rawdata[[
    #Time Stamp
    'Begin Time',
    #BTS and Cells identifiers
    'eNodeB Name',
    'E-UTRAN FDD Cell Name',
    # Target variables (choose one for Y)
    'PS Traffic Volume(GB)_ITBBU&SDR',
    # Key traffic load indicators
    'UL PRB Utilization Rate(%)',
    'DL PRB Available (Bandwidth)',
    'Mean Number of RRC Connection User',
    'Maximum Active User Number on User Plane',
    # Quality KPIs
    'RRC Establishment Success Rate(%)',
    'E-RAB Setup Success Rate(%)',
    'E-RAB Drop Rate(%)',
    'RRC Drop Rate(%)',
    'Cell Uplink BLER(%)',
    'Cell Downlink BLER(%)',
    # Signal quality affecting throughput
    'DL Average MCS',
    'UL Average MCS',
    'Average CQI(N/A)',
    # Coverage and Interference
    'LTE Average TA(km)',
    'Average Cell RSSI(dBm)',
    # Mobility
    'Success Rate of Outgoing Handover(Cell)(%)',
    'Success Rate of Intra-RAT Inter-frequency Cell Outgoing Handover(%)',
    'Number of Ping-Pong Handover']].rename(columns={
    # Time Stamp
    'Begin Time': 'timestamp',
    # Target Variables
    'PS Traffic Volume(GB)_ITBBU&SDR': 'ps_traffic_volume_gb',
    # BTS and Cell Identifiers
    'eNodeB Name': 'enodeb_name',
    'E-UTRAN FDD Cell Name': 'cell_name',
    # Key traffic load indicators
    'UL PRB Utilization Rate(%)': 'ul_prb_util_%',
    'DL PRB Available (Bandwidth)': 'dl_prb_available_bandwidth',
    'Mean Number of RRC Connection User': 'mean_no_rrc_users',
    'Maximum Active User Number on User Plane': 'max_active_no_users_uplane',
    # Quality KPIs
    'RRC Establishment Success Rate(%)': 'rrc_success_rate_%',
    'E-RAB Setup Success Rate(%)': 'erab_setup_success_rate_%',
    'E-RAB Drop Rate(%)': 'erab_drop_rate_%',
    'RRC Drop Rate(%)': 'rrc_drop_rate_%',
    'Cell Uplink BLER(%)': 'cell_ul_bler_rate_%',
    'Cell Downlink BLER(%)': 'cell_dl_bler_rate_%',
    # RF/PHY Layer
    'DL Average MCS': 'dl_avg_mcs',
    'UL Average MCS': 'ul_avg_mcs',
    'Average CQI(N/A)' : 'avg_cqi',
    # Coverage and Interference
    'LTE Average TA(km)': 'avg_ta_km',
    'Average Cell RSSI(dBm)': 'avg_rssi_dbm',
    # Mobility
    'Success Rate of Outgoing Handover(Cell)(%)': 'outgoing_ho_success_rate_%',
    'Success Rate of Intra-RAT Inter-frequency Cell Outgoing Handover(%)': 'intra_rat_ho_success_rate_%',
    'Number of Ping-Pong Handover': 'no_ping_pong_ho_count'
})
# ==============================================================================
# 🔍  Exploring lte_ps_traffic EDA Summary
# ==============================================================================
basic_EDA_summary(lte_ps_traffic, name = 'lte ps traffic')

# 🔍 checking how many LTE enodeB
lte_ps_traffic['enodeb_name'].unique()
lte_ps_traffic['enodeb_name'].nunique()
lte_ps_traffic['enodeb_name'].value_counts()
# 🔍 checking how many LTE Cells
lte_ps_traffic['cell_name'].unique()
lte_ps_traffic['cell_name'].nunique()
lte_ps_traffic['cell_name'].value_counts().to_csv('exports/ltepstraffi_countofcells.csv')

# ==============================================================================
# ⚙️️ Summary statistics Function: Computes descriptive statistics for all numeric columns and exports to Excel
# ==============================================================================
def descriptive_statistics_numeric(df, name="DataFrame"):
    numeric_df = df.select_dtypes(include='number')
    summary_stats = pd.DataFrame({
        'Min': numeric_df.min().round(2),
        'Max': numeric_df.max().round(2),
        'Mean': numeric_df.mean().round(2),
        'Std. Deviation': numeric_df.std().round(2),
        'Variance': numeric_df.var().round(2),
        'Skewness': numeric_df.skew().round(2),
        'Kurtosis': numeric_df.kurtosis().round(2),
        'Sum': numeric_df.sum().round(2).round(2),
        'Median': numeric_df.median().round(2),
        'Missing': numeric_df.isna().sum().round(2),
        'Row count': len(numeric_df)
    })
    summary_stats.index.name = 'Feature'
    summary_stats.reset_index(inplace=True)
    file_path = "exports/Libyana LTE KPIs/" + name + "_numeric_summary.xlsx"
    summary_stats.to_excel(file_path, index=False)
    print("\n🟢 Exported numeric summary for '" + name + "' to: " + file_path)
    return summary_stats

# ==============================================================================
# 🔍 conduct Descriptive analysis for lte_ps_traffic
# ==============================================================================
descriptive_statistics_numeric(lte_ps_traffic, name ='lte_ps_traffic_1')

# Histogram to show skewness
def histogram_plot(df, kpi_name, data_set):
    df[kpi_name].plot(
        kind='hist',
        bins=30,
        figsize=(12, 6),
        edgecolor='black',
        color='skyblue',
        alpha=0.7,
        title=f'Histogram of {kpi_name} for {data_set}')
    plt.xlabel(f'{kpi_name}')
    plt.ylabel('Frequency')
    plt.grid(axis='y', alpha=0.3)
    plt.savefig(f'exports/Libyana LTE KPIs/ds/{data_set}_histogram of {kpi_name}.png')
    plt.show()
#
histogram_plot(lte_ps_traffic,'ps_traffic_volume_gb')
histogram_plot(lte_ps_traffic,'mean_no_rrc_users')
histogram_plot(lte_ps_traffic,'erab_setup_success_rate_%')
histogram_plot(lte_ps_traffic,'rrc_success_rate_%')
histogram_plot(lte_ps_traffic,'rrc_drop_rate_%')

histogram_plot(TRI022L,'ps_traffic_volume_gb', 'TRI022L')

# ==============================================================================
#  Group by timestamp and enodeb_name, and aggregate using appropriate functions
# ==============================================================================
#DataFrame
agg_sites_traffic = lte_ps_traffic.groupby(['timestamp', 'enodeb_name']).agg({
    'ps_traffic_volume_gb': 'sum',
    'ul_prb_util_%': 'mean',
    'dl_prb_available_bandwidth': 'mean',
    'mean_no_rrc_users': 'sum',
    'max_active_no_users_uplane': 'sum',
    'rrc_success_rate_%': 'mean',
    'erab_setup_success_rate_%': 'mean',
    'erab_drop_rate_%': 'mean',
    'rrc_drop_rate_%': 'mean',
    'cell_ul_bler_rate_%': 'mean',
    'cell_dl_bler_rate_%': 'mean',
    'dl_avg_mcs': 'mean',
    'ul_avg_mcs': 'mean',
    'avg_cqi': 'mean',
    'avg_ta_km': 'mean',
    'avg_rssi_dbm': 'mean',
    'outgoing_ho_success_rate_%': 'mean',
    'intra_rat_ho_success_rate_%': 'mean',
    'no_ping_pong_ho_count':'sum'
}).reset_index()

# ==============================================================================
# 🔍 Basic EDA Exploration from aggregated data
# ==============================================================================
basic_EDA_summary(agg_sites_traffic, name ='site level data')
# 🔍 conduct Descriptive analysis for aggregated eNodeB sites Traffic
descriptive_statistics_numeric(agg_sites_traffic, name ='agg_sites_traffic')

#Site Level Descriptive Statistics
descriptive_statistics_numeric(agg_sites_traffic[agg_sites_traffic['enodeb_name']=='TRI022L'],
                               name='TRI022')
# ==============================================================================
# DATA PREPROCESSING of Aggregated Sites Traffic
# ==============================================================================
# 1. Data formatting (DateTime type conversinon & indexing)
# ==============================================================================
agg_sites_traffic.info()
agg_sites_traffic.head()

# Changing Timestamp to Datetime format
agg_sites_traffic['timestamp'] = pd.to_datetime(agg_sites_traffic['timestamp'])
# Setting the timestamp as index for TSA
agg_sites_traffic.set_index('timestamp', inplace=True)
# eNodeB column correct format from object type to string type for clarity
agg_sites_traffic['enodeb_name'] = agg_sites_traffic['enodeb_name'].astype('string')

#Exporting the formated and organised data
agg_sites_traffic.to_excel('exports/Libyana LTE KPIs/aggregated_sites_traffic.xlsx')
# ==============================================================================
# 2. Missing Data
# ==============================================================================
# since number of missing data
agg_sites_traffic.shape
agg_sites_traffic=agg_sites_traffic.dropna()
agg_sites_traffic.shape

#no_pingpong column contain in the original lte set
agg_sites_traffic['no_ping_pong_ho_count'].isnull().count()
agg_sites_traffic.drop(columns=['no_ping_pong_ho_count'], inplace= True)
agg_sites_traffic.info()

# ==============================================================================
# eNodeB subsetting
# ==============================================================================
# list the enodebs
agg_sites_traffic['enodeb_name'].unique()

# Creating a function to subset individual enodeBs
def subset_enodeb (df,name):
    return df[df['enodeb_name']==name]

# Creating a datafram for each eNodeB
TRI022L = subset_enodeb(agg_sites_traffic, name='TRI022L')
TRI055L = subset_enodeb(agg_sites_traffic, name='TRI055L')
TRI1007L = subset_enodeb(agg_sites_traffic, name='TRI1007L')
TRI165L = subset_enodeb(agg_sites_traffic, name='TRI165L')
TRI166L = subset_enodeb(agg_sites_traffic, name='TRI166L')
TRI183L = subset_enodeb(agg_sites_traffic, name='TRI183L')
TRI194L = subset_enodeb(agg_sites_traffic, name='TRI194L')
TRI209L = subset_enodeb(agg_sites_traffic, name='TRI209L')
TRI231L = subset_enodeb(agg_sites_traffic, name='TRI231L')
TRI435L = subset_enodeb(agg_sites_traffic, name='TRI435L')
TRI695L = subset_enodeb(agg_sites_traffic, name='TRI695L')
TRI730L = subset_enodeb(agg_sites_traffic, name='TRI730L')
TRI809L = subset_enodeb(agg_sites_traffic, name='TRI809L')
TRI825L = subset_enodeb(agg_sites_traffic, name='TRI825L')
TRI878L = subset_enodeb(agg_sites_traffic, name='TRI878L')
TRI882L = subset_enodeb(agg_sites_traffic, name='TRI882L')
# --------------------------------------------------------
# Creating eNodeB name list (useful for for-loops)
# eNodeB
enodeb_names = [
    'TRI022L', 'TRI055L', 'TRI1007L', 'TRI165L', 'TRI166L', 'TRI183L', 'TRI194L', 'TRI209L',
    'TRI231L', 'TRI435L', 'TRI695L', 'TRI730L', 'TRI809L', 'TRI825L', 'TRI878L', 'TRI882L']

# ==============================================================================
# 3. CORRELATION ANALYSIS
# ==============================================================================
# ****** Correlation analysis - Per site level
## Computes spearman correlation (linear relationship) Range: [-1, 1]
corr_TRI022L =TRI022L.select_dtypes(include='number').corr(method='spearman')

# -------
# >> 3.1 Creating a correlation Function
def calculate_corr(df, name):
    filtered_df = df[df['enodeb_name'] == name]
    numeric_df = filtered_df.select_dtypes(include='number')
    correlation_matrix = numeric_df.corr(method='spearman')
    return correlation_matrix
# -------
# Calculate the Spearman correlation for eNodeBs and exporting the results to excel file
with pd.ExcelWriter('exports/Libyana LTE KPIs/Correlation/corr_eNodeBs.xlsx', engine='xlsxwriter') as writer:
    for name in enodeb_names:
        corr_matrix = calculate_corr(agg_sites_traffic, name)
        corr_matrix.to_excel(writer, sheet_name=name[:31])
# -------
# >> 3.3 Correlation Heatmap Visualisation and Exporting results into local disk
    for name in enodeb_names:
        corr_matrix = calculate_corr(agg_sites_traffic, name=name)
        plt.figure(figsize=(7, 6))
        sns.heatmap(
            corr_matrix,
            annot=True,
            fmt='.2f',
            cmap='coolwarm',
            square=True,
            cbar_kws={'shrink': 0.5},
            annot_kws={'fontsize': 6})
        plt.title(f'Pair-wise Correlation Matrix for {name}')
        plt.xticks(rotation=45, fontsize=8, ha='right')
        plt.yticks(fontsize=8)
        plt.xlabel('KPI Features')
        plt.ylabel('KPI Features')
        plt.tight_layout()
        plt.savefig(f'exports/Libyana LTE KPIs/Correlation/Plots/correlation_matrix_{name}.png', dpi=300)
        plt.show()